In [2]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 115.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.2 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import copy
%matplotlib inline

In [6]:
df = pd.read_csv('./Ethos_Dataset_Multi_Label.csv',sep=';')
df.columns

Index(['comment', 'violence', 'directed_vs_generalized', 'gender', 'race',
       'national_origin', 'disability', 'religion', 'sexual_orientation'],
      dtype='object')

In [7]:
labels = ['violence', 'directed_vs_generalized', 'gender', 'race', 'national_origin', 'disability', 'religion', 'sexual_orientation']
text = df['comment']
label_df = df[labels]


In [8]:
def transform_labels(record):
    record = copy.deepcopy(record)
    _max  = record.argmax()
    for l, v in record.items():
        if l != record.index[_max]:
            record[l] = 0
        else:
            record[l] = 1
    return record

l_tmp = label_df.apply(transform_labels, axis=1)

In [9]:
l_tmp

,violence,directed_vs_generalized,gender,race,national_origin,disability,religion,sexual_orientation
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
428,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
429,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
430,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [23]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
from tensorflow.keras.layers import Input, Dense, Activation, Flatten
from tensorflow.keras.models import Model

In [11]:
# Load pre-trained BERT model
bert_model = TFBertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [12]:
max_sequence_length = 128

In [13]:
def preprocess_text_data(text_data, max_sequence_length):
    input_ids = []
    for text in text_data:
        tokens = tokenizer(text, padding='max_length', truncation=True, max_length=max_sequence_length, return_tensors="tf")['input_ids'][0]
        input_ids.append(tokens)
    return tf.convert_to_tensor(input_ids)

In [14]:
text_tokenized = preprocess_text_data(text, max_sequence_length)

In [15]:
text_tokenized.numpy()

array([[  101,  2017,  2323, ...,     0,     0,     0],
       [  101,  2017,  2298, ...,     0,     0,     0],
       [  101,  2017,  2298, ...,     0,     0,     0],
       ...,
       [  101,  8273,  4830, ...,     0,     0,     0],
       [  101,  1037, 24004, ...,     0,     0,     0],
       [  101,  1006,  1006, ...,     0,     0,     0]], dtype=int32)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(text_tokenized.numpy(), l_tmp, test_size=0.2, random_state=42)

In [17]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((346, 128), (87, 128), (346, 8), (87, 8))

In [18]:
 y_train.shape

(346, 8)

In [28]:
## Model archi

num_classes = 8

input_text = Input(shape=(max_sequence_length,), dtype=tf.int32)

# Get BERT model's output
bert_output = bert_model(input_text)[0]

# Freeze BERT layers
for layer in bert_model.layers:
    layer.trainable = False

# Add fully connected layers with tanh activation
x = Dense(256, activation='tanh')(bert_output)
x = Flatten()(Dense(128, activation='tanh')(x))

# Add the output layer for classification
output = Dense(num_classes, activation='softmax')(x)

# Create a Keras model
model = Model(inputs=input_text, outputs=output)

# Compile the model with appropriate loss and optimizer
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [29]:
epochs=10
batch_size=32
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/10
11/11 [==============================] - 17s 279ms/step - loss: 4.3149 - accuracy: 0.1474
Epoch 2/10
11/11 [==============================] - 3s 279ms/step - loss: 2.5993 - accuracy: 0.2457
Epoch 3/10
11/11 [==============================] - 3s 280ms/step - loss: 1.9907 - accuracy: 0.3179
Epoch 4/10
11/11 [==============================] - 3s 284ms/step - loss: 1.3090 - accuracy: 0.5838
Epoch 5/10
11/11 [==============================] - 3s 284ms/step - loss: 1.0238 - accuracy: 0.7254
Epoch 6/10
11/11 [==============================] - 3s 279ms/step - loss: 0.9201 - accuracy: 0.7197
Epoch 7/10
11/11 [==============================] - 3s 280ms/step - loss: 0.6464 - accuracy: 0.8584
Epoch 8/10
11/11 [==============================] - 3s 281ms/step - loss: 0.5545 - accuracy: 0.8728
Epoch 9/10
11/11 [==============================] - 3s 291ms/step - loss: 0.3847 - accuracy: 0.9393
Epoch 10/10
11/11 [==============================] - 3s 282ms/step - loss: 0.2972 - accuracy: 0.974